In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# import necessary packages
import os
import numpy as np
import pandas as pd

from skimage import morphology

from vis_utils import *
from measure_utils import *
from data_utils import cnt_polygon_test

In [ ]:
DIR_TEST = "/Users/jinzhou/Desktop/USCAP/test" # parent folder for input and gt in testset
DIR_DL = "/Users/jinzhou/Desktop/USCAP/dl_output" # folder for DL predictions on testset
DIR_SAVE_RESULTS = "/Users/jinzhou/Desktop/USCAP/results" # folder for saving analysis results

In [ ]:
def clean(img):
    # removing small holes and objects
    small_holes_th = np.count_nonzero(img) / 20
    small_objects_th = np.count_nonzero(img) / 20
    img_rmh = morphology.remove_small_holes(img, area_threshold=small_holes_th)
    img_rmo = morphology.remove_small_objects(img_rmh, min_size =small_objects_th)
    res = np.uint8(img_rmo*255)
    return res

def get_largest_cnt(contours):
    largest_contour = None
    max_area = 0
    max_area_index = 0
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area > max_area:
            largest_contour = contour
            max_area = area
            max_area_index = i
    return largest_contour, max_area_index

In [ ]:
def get_three_cnts(img):
    img_outer = (img > 0) * 255 # area under the outer contour of media
    img_middle = (img > 85) * 255 # area under the outer contour of intima
    img_inner = (img > 170) * 255 # area under the outer contour of lumen
#     imshow_k_in_row([gt, img, img_outer, img_middle, img_inner])
    img_outer = clean(img_outer)
    img_middle = clean(img_middle)
    img_inner = clean(img_inner)
#     imshow_k_in_row([gt, img, img_outer, img_middle, img_inner])
    cnt_outers, _ = cv2.findContours(img_outer.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt_outer, _ = get_largest_cnt(cnt_outers)
    cnt_outer = cnt_outer.squeeze()
    cnts_middle, _ = cv2.findContours(img_middle.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts_middle = [cnt.squeeze() for cnt in cnts_middle]
    cnts_inner, _ = cv2.findContours(img_inner.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts_inner = [cnt.squeeze() for cnt in cnts_inner]
    
    return cnt_outer, cnts_middle, cnts_inner

In [ ]:
# df_features_label = pd.read_csv(os.path.join(DIR_SAVE_RESULTS, "features_label_dl.csv"))
# print(df_features_label.head())

In [ ]:
df = pd.DataFrame(columns = ['WSI_ID', 'Artery_ID', 'Area', 'Thickness_Media_Abs', 
                             'Thickness_Intima_Abs', 'Thickness_Wall_Abs'])

path_to_save = os.path.join(DIR_SAVE_RESULTS, "thickness_dl.csv")
fails = []
for img_name in os.listdir(DIR_DL):
#     if not img_name in ["11_26609_000_011_L02_TRI_A11_pred.png",
#                         "11_26609_000_011_L02_TRI_A08_pred.png", 
#                         "11_26609_020_006 A15 TRI_A16_pred.png",
#                         "11_26609_020_006 A15 TRI_A22_pred.png"]:
#         continue
    
    if not img_name.endswith("pred.png"): continue
    # example img_name: 11_26609_000_011_L02_TRI_A01_pred.png
    wsi_id = img_name[:-13] # eg. 11_26609_000_011_L02_TRI
    artery_id = img_name[-12: -9] # eg. A01
    gt_name = img_name[:-8] +"gt.png" # eg. 11_26609_000_011_L02_TRI_A01_gt.png
#     print(img_name)
#     row = df_features_label.loc[df_features_label.loc[:, "WSI_Artery_ID"] == img_name[:-9]]
#     print("Label", row["Label"].values[0], 
#           "Media Average", round(row["Media Average"].values[0], 2), 
#           "Media Power", round(row["Media Power"].values[0], 2), 
#           "Intima Average", round(row["Intima Average"].values[0], 2), 
#           "Intima Power", round(row["Intima Power"].values[0], 2))
    input_path = os.path.join(DIR_TEST, "input", img_name[:-9]+".png")
    gt_path = os.path.join(DIR_DL, gt_name)
    pred_path = os.path.join(DIR_DL, img_name)
    
    input_img = cv2.cvtColor(cv2.imread(input_path), cv2.COLOR_BGR2RGB)
    gt = cv2.cvtColor(cv2.imread(gt_path), cv2.COLOR_BGR2GRAY)
    pred = cv2.cvtColor(cv2.imread(pred_path), cv2.COLOR_BGR2GRAY)
    
    # all media/lumen/intima have to exist to measure thickness
    if len(np.unique(pred)) != 4: 
        fails.append(img_name)
        continue
    
    cnt_outer, cnts_middle, cnts_inner = get_three_cnts(pred)
    # vis contours on top of an artery
    input_w_cnts = plot_artery_ann(input_img, cnt_outer, cnts_middle, cnts_inner)
    imshow_k_in_row([input_w_cnts])
    # vis only contours
    h, w = input_img.shape[:2]
    vis_cnts = np.zeros((h, w, 3), np.uint8)
    vis_cnts = plot_artery_ann(vis_cnts, cnt_outer, cnts_middle, cnts_inner)
    exist = False
    for idx, cnt_middle in enumerate(cnts_middle):
        for idy, cnt_inner in enumerate(cnts_inner):
            if cnt_polygon_test(cnt_inner, cnt_middle):
                area_inner = cv2.contourArea(cnt_inner)
                # radial lines intersecs with excludde should be discarded
                exclude = cnts_middle[:idx] + cnts_middle[idx+1:]+ cnts_inner[:idy] + cnts_inner[idy+1:]
                thick_media, thick_intima, thick_wall = measure_thickness(wsi_id, artery_id, cnt_outer, cnt_middle, cnt_inner,
                                                            vis_cnts, angle_width=10, exclude=exclude)
                row = {'WSI_ID':wsi_id, 'Artery_ID':artery_id+'_'+str(idx)+'_'+str(idy), 
                       'Area': area_inner, 'Thickness_Media_Abs':thick_media, 'Thickness_Intima_Abs':thick_intima,
                       'Thickness_Wall_Abs':thick_wall}
                df = df.append(row, ignore_index=True)
                exist = True
    if not exist:
        cnt_middle, idx = get_largest_cnt(cnts_middle)
        cnt_inner, idy = get_largest_cnt(cnts_inner)
        area_inner = cv2.contourArea(cnt_inner)
        # radial lines intersecs with excludde should be discarded
        exclude = cnts_middle[:idx] + cnts_middle[idx+1:]+ cnts_inner[:idy] + cnts_inner[idy+1:]
        thick_media, thick_intima, thick_wall = measure_thickness(wsi_id, artery_id, cnt_outer, cnt_middle, cnt_inner,
                                                    vis_cnts, angle_width=10, exclude=exclude)
        row = {'WSI_ID':wsi_id, 'Artery_ID':artery_id+'_'+str(idx)+'_'+str(idy), 
               'Area': area_inner, 'Thickness_Media_Abs':thick_media, 'Thickness_Intima_Abs':thick_intima,
               'Thickness_Wall_Abs':thick_wall}
        df = df.append(row, ignore_index=True)
    imshow_k_in_row([vis_cnts])
# df.to_csv(path_to_save, index=False)

In [ ]:
print(fails)

In [ ]:
df.head()

In [ ]:
row["Media Average"].values[0]